# Deep backward schemes for high-dimensional nonlinear PDEs

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import sys

sys.path.append("..")

In [2]:
from dbdp.solvers import DBDP1Solver
from dbdp import DBDPModel

$$
\begin{cases}
    \partial_t u + \mathcal{L}u + f(.,.,u,\sigma\cdot Du) = 0 & \text{on } [0,T)\times\mathbb{R}^d, \\
    u(T,.) = g & \text{on }\mathbb{R}^d.
\end{cases}
$$

## Example 1 :

$$
    \mu=0.2,\ \sigma=1,\ T=2,\ x_0=1
$$

$$
    g(x)=\cos(x)
$$

$$
    f(t,x,y,z)=(\cos(x)(e^{\frac{T-t}{2}}+\sigma^2/2)+\mu\sin(x))e^{\frac{T-t}{2}}-\frac{1}{2}(\sin(x)\cos(x)e^{T-t})^2+\frac{1}{2}(yz)^2
$$

In [3]:
class Example1Model(DBDPModel):
    def __init__(self, mu: float, sigma: float, maturity: float):
        self._mu = torch.tensor(mu)
        self._sigma = torch.tensor(sigma)
        self._maturity = maturity

    def drift(self, t, x) -> torch.Tensor:
        return self._mu

    def diffusion(self, t, x) -> torch.Tensor:
        return self._sigma

    def f(self, t: float, x: torch.Tensor, y: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
        a = torch.cos(x) * (np.exp((self._maturity - t) / 2) + self._sigma**2 / 2) + self._mu * torch.sin(x)
        b = -((torch.sin(x) * torch.cos(x) * np.exp(self._maturity - t)) ** 2) + (y * z) ** 2
        return a * np.exp((self._maturity - t) / 2) + b / 2

    def g(self, x: torch.Tensor) -> torch.Tensor:
        return torch.cos(x)


mu = 0.2
sigma = 1.0
maturity = 2.0
x0 = torch.Tensor([1.0])  # So that x0.dim() == 1
dim = x0.size(0)

example1_model = Example1Model(mu, sigma, maturity)
example_true_u = lambda t, x: np.exp((maturity - t) / 2) * torch.cos(x)

time_steps = 240
dt = maturity / time_steps
sample_count = 10_000

In [4]:
example1_dbdp1_solver = DBDP1Solver(example1_model, maturity, time_steps, dim)
# example1_dbdp1_solver.load("../models/example-01.pt")

In [5]:
# Generating datas
x_paths, dw = example1_model.generate_datas(x0, dt, time_steps, dim, sample_count)

print("Shapes:")
print(f"-      dw: {dw.shape}")
print(f"- x_paths: {x_paths.shape}")

Shapes:
-      dw: torch.Size([10000, 240, 1])
- x_paths: torch.Size([10000, 241, 1])


In [ ]:
trains_losses, tests_losses = example1_dbdp1_solver.train(
    x_paths,
    dw,
    num_epochs=400,
    batch_size=1_000,
    lr=1e-2,
)
# example1_dbdp1_solver.save("../models/example-01.pt")

In [ ]:
u_approx = example1_dbdp1_solver


time = np.linspace(0, maturity, time_steps + 1)
t = time[235]
xs = x_paths[:, 235].unsqueeze(1)[:, :, 0]
ys = u_approx(t, xs).detach()

plt.figure(figsize=(12, 6), dpi=200)
plt.title(f"t={t}")
plt.plot(xs, ys, ".", markersize=4, label="Numeric", alpha=0.5)
plt.plot(xs, example_true_u(t, xs), ".", markersize=4, label="Analytic", alpha=0.5)
plt.grid(alpha=0.5, ls="dotted")
plt.xlabel("$x$")
plt.ylabel("$u(t,x)$")
plt.tight_layout()
plt.legend(loc="upper left")
plt.show()

In [ ]:
def plot_losses(train_losses, test_losses):
    plt.figure(figsize=(12, 6), dpi=200)
    plt.title("Evolution of training and test error")
    plt.plot(train_losses, label="Train loss")
    plt.plot(test_losses, label="Test loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss (MSE)")
    plt.loglog()
    plt.grid(alpha=0.5, ls="dotted")
    plt.tight_layout()
    plt.legend()

In [ ]:
plot_losses(trains_losses[0], tests_losses[0])